In [1]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util
import numpy as np
import pandas as pd

In [2]:
config_path = r'model\detect_license_plate\pipeline.config'
label_path = r'config\label_map.pbtxt'
checkpoint_path = r'model\detect_license_plate\checkpoint\ckpt-0'
MAIN_FOLDER_PATH = os.path.dirname(os.getcwd())

In [3]:
MAIN_FOLDER_PATH

'D:\\Code_school_nam3ki2\\TestModel'

In [11]:
category_index = label_map_util.create_category_index_from_labelmap(os.path.join(MAIN_FOLDER_PATH, label_path))
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(os.path.join(MAIN_FOLDER_PATH, config_path))
detection_model = model_builder.build(model_config=configs['model'], is_training=False)
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(MAIN_FOLDER_PATH, checkpoint_path)).expect_partial()

In [52]:
dummy_input = tf.random.uniform((1, 320, 320, 3))

dummy_output = detection_model(dummy_input)
total_params = 0
total_train_params = 0
total_untrain_params = 0
layer_cnt = 0
list_layer = []
list_input_shape = []
list_param = []

In [53]:
for var in detection_model.variables:
    layer_cnt += 1
    num_params = np.prod(var.shape)
    total_params += num_params
    if var.trainable:
        total_train_params += num_params
    else:
        total_untrain_params += num_params
    list_layer.append(var.name)
    list_input_shape.append(var.shape)
    list_param.append(num_params)
#Tạo bảng với 3 cột: Layer, Input Shape, Parameters sử dụng thư viện vẽ bảng trong terminal
# Create a DataFrame with the information
data = {
    "Layer": list_layer,
    "Input Shape": list_input_shape,
    "Parameters": list_param
}
df = pd.DataFrame(data)
df.style.set_properties(**{'text-align': 'left'})
pd.set_option('display.max_rows', None)
print(df.to_string(index=False))
print("-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------")
print("Total Layers: ", layer_cnt)
print("Total Parameters: ", total_params)
print("Total Trainable Parameters: ", total_train_params)
print("Total Untrainable Parameters: ", total_untrain_params)

                                                                                                                             Layer       Input Shape  Parameters
            ssd_meta_arch_1/WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead/BoxPredictor/depthwise_kernel:0    (3, 3, 128, 1)        1152
            ssd_meta_arch_1/WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead/BoxPredictor/pointwise_kernel:0   (1, 1, 128, 24)        3072
                        ssd_meta_arch_1/WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalBoxHead/BoxPredictor/bias:0              (24)          24
        ssd_meta_arch_1/WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/ClassPredictor/depthwise_kernel:0    (3, 3, 128, 1)        1152
        ssd_meta_arch_1/WeightSharedConvolutionalBoxPredictor/WeightSharedConvolutionalClassHead/ClassPredictor/pointwise_kernel:0   (1, 1, 128, 12)        1536
                    ssd_meta_arch_

In [38]:
# Write the tabulate string to a text file
with open('Model_Structure.txt', 'w') as f:
    f.write(df.to_string(index=False))